In [1]:
import sys
sys.path.append("../src")

from data_loader import WildFireDataLoader
from model_loader import ModelLoader

import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import pandas as pd
import numpy as np
from pathlib import Path

1. Загрузка данных:

In [ ]:
loader = WildFireDataLoader(Path("../data/raw_orig"))
data_dict = loader.load_all_data()

train_df = data_dict["train"]
test_df = data_dict["test"]
metadata = data_dict["metadata"]

print(f"Тренировочные данные: {len(train_df)} записей")
print(f"Тестовые данные: {len(test_df)} записей")
print(f"Метаданные: {metadata}")

INFO:data_loader:Загрузка данных...
INFO:data_loader:Поиск данных в: ../data/raw
INFO:data_loader:Структура: {'train_exists': True, 'test_exists': True, 'train_subfolders': ['nowildfire', 'wildfire'], 'test_subfolders': ['nowildfire', 'wildfire'], 'image_formats': {<built-in method lower of str object at 0x73888db25740>, <built-in method lower of str object at 0x73888db25770>, <built-in method lower of str object at 0x73888db257a0>, <built-in method lower of str object at 0x73888db277e0>, <built-in method lower of str object at 0x73888db27600>, <built-in method lower of str object at 0x73888db27630>, <built-in method lower of str object at 0x73888db25680>, <built-in method lower of str object at 0x73888db27570>, <built-in method lower of str object at 0x73888db255f0>, <built-in method lower of str object at 0x73888db274b0>, <built-in method lower of str object at 0x73888db27390>, <built-in method lower of str object at 0x73888db273c0>, <built-in method lower of str object at 0x73888db2

Тренировочные данные: 30250 записей
Тестовые данные: 6300 записей
Метаданные: {'train_samples': 30250, 'test_samples': 6300, 'train_classes': {0: 15750, 1: 14500}, 'test_classes': {0: 3480, 1: 2820}, 'structure': {'train_exists': True, 'test_exists': True, 'train_subfolders': ['nowildfire', 'wildfire'], 'test_subfolders': ['nowildfire', 'wildfire'], 'image_formats': {<built-in method lower of str object at 0x73888db25740>, <built-in method lower of str object at 0x73888db25770>, <built-in method lower of str object at 0x73888db257a0>, <built-in method lower of str object at 0x73888db277e0>, <built-in method lower of str object at 0x73888db27600>, <built-in method lower of str object at 0x73888db27630>, <built-in method lower of str object at 0x73888db25680>, <built-in method lower of str object at 0x73888db27570>, <built-in method lower of str object at 0x73888db255f0>, <built-in method lower of str object at 0x73888db274b0>, <built-in method lower of str object at 0x73888db27390>, <bu

2. Загрузка моделей:

In [6]:
model_loader = ModelLoader("cuda")
models_dict = model_loader.load_all_models()

for key, info in model_loader.get_model_info().items():
    print(f"\n  {key}: {info['name']}")
    print(f"    Параметров: {info['num_parameters']:,}")
    print(f"    Описание: {info['description']}")

INFO:model_loader:Используется устройство: cuda
INFO:model_loader:Загрузка всех моделей...
INFO:model_loader:Загрузка модели: Gurveer05/vit-base-patch16-224-in21k-fire-detection
INFO:model_loader:Модель base успешно загружена. Параметров: 85,800,194
INFO:model_loader:Загрузка модели: EdBianchi/vit-fire-detection
INFO:model_loader:Модель finetuned успешно загружена. Параметров: 85,800,963
INFO:model_loader:Загружено 2 из 2



  base: Gurveer05/vit-base-patch16-224-in21k-fire-detection
    Параметров: 85,800,194
    Описание: Базовая ViT модель, дообученная на датасете пожаров

  finetuned: EdBianchi/vit-fire-detection
    Параметров: 85,800,963
    Описание: Дообученная версия ViT с высокими метриками


Проверим предобработку на примере:

In [5]:
sample_image_path = train_df.iloc[0]['image_path']
sample_image = Image.open(sample_image_path)

print(f"Исходное изображение: {sample_image.size}")
print(f"Класс: {train_df.iloc[0]['class_name']}")

for model_key in ['base', 'finetuned']:
    try:
        inputs = model_loader.preprocess_batch(sample_image, model_key)
        print(f"\nМодель '{model_key}':")
        print(f"  Размеры тензоров: {inputs['pixel_values'].shape}")
        print(f"  Диапазон значений: [{inputs['pixel_values'].min():.3f}, {inputs['pixel_values'].max():.3f}]")
    except Exception as e:
        print(f"  Ошибка: {e}")

Исходное изображение: (350, 350)
Класс: wildfire

Модель 'base':
  Размеры тензоров: torch.Size([1, 3, 224, 224])
  Диапазон значений: [-1.000, 0.608]

Модель 'finetuned':
  Размеры тензоров: torch.Size([1, 3, 224, 224])
  Диапазон значений: [-1.000, 0.608]
